In [63]:
from glob import glob
import numpy as np
import cv2

In [65]:
commands = ["black","cancel","centeralign","copy","large","medium","newslide","paste","red","textbox"]
def crop(buffer, clip_len):
    # randomly select time index for temporal jittering
    time_index = np.random.randint(buffer.shape[1] - clip_len)
    # randomly select start indices in order to crop the video

    # crop and jitter the video using indexing. The spatial crop is performed on 
    # the entire array, so each frame is cropped in the same location. The temporal
    # jitter takes place via the selection of consecutive frames
    buffer = buffer[:,time_index:time_index + clip_len,:,:]

    return buffer   

In [70]:
dataset = dict()
for c in commands:
    dataset[c] = []
    for f in sorted(glob("./100/test/"+c+"/*")):
        capture = cv2.VideoCapture(f)
        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        sample = np.linspace(0, frame_count.shape[0]-1, num=16, dtype=int)
        if frame_count>32:

            buffer = np.empty((16,80,100, 3), np.dtype('float32'))
            count = 0
            retaining = True
            j=0
            # read in each frame, one at a time into the numpy buffer array
            while (count < frame_count and retaining):
                retaining, frame = capture.read()
                if count == sample[j]:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    buffer[i] = frame
                    j+=1
                count += 1
            buffer = buffer.transpose((3, 0, 1, 2)) 
            buffer = (buffer - 128)/128
            dataset[c].append(buffer)


In [72]:
tr_path="/Users/soshiyuu/Devlopment/Github/amad/PyTorch_Speaker_Verification/train_tisv/"
ts_path="/Users/soshiyuu/Devlopment/Github/amad/PyTorch_Speaker_Verification/test_tisv/"
i=0
for c in commands:
    for i in range(len(dataset[c])):
        np.save(ts_path+c+"/"+c+"%d"%i,dataset[c][i])


In [56]:
print(np.array(np.stack(dataset["black"]).shape))

[ 88   3  32  80 100]


In [13]:
for c in commands:
    for i in range(len(dataset[c])):
        dataset[c][i] = np.resize(np.array(dataset[c][i]),(70,80))


In [25]:
os.listdir(tr_path)


['.DS_Store',
 'speaker8.npy',
 'speaker9.npy',
 'speaker10.npy',
 'speaker4.npy',
 'speaker5.npy',
 'speaker7.npy',
 'speaker6.npy',
 'speaker2.npy',
 'speaker3.npy',
 'speaker1.npy']

In [85]:
import random
utter_num = 5
path = "/Users/soshiyuu/Devlopment/Github/amad/PyTorch_Speaker_Verification/train_tisv/"
np_file_dir = [b for b in os.listdir(path) if b[0] != "."]
selected_dir = random.sample(np_file_dir, 1)[0]  # select random speaker          
np_file_list = os.listdir(os.path.join(path+selected_dir))
selected_file = random.sample(np_file_list,utter_num)

print(selected_file)

['paste33.npy', 'paste24.npy', 'paste73.npy', 'paste13.npy', 'paste48.npy']


In [ ]:
utters = np.load(os.path.join(path, selected_file))        # load utterance spectrogram of selected speaker
utter_index = np.random.randint(0, utters.shape[0], self.utter_num)   # select M utterances per speaker
utterance = utters[utter_index]     